In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score
!pip install xgboost
from xgboost import XGBClassifier
import numpy as np

In [2]:
df = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/FFPE_Discovery_Chowdhury/FFPE_Discovery64_normalized.csv",header=0,low_memory=False)
df.head()

,Row.names,ABCB8,ACADSB,ACOT7,AIFM1,ARNT,ATM,ATP6V1D,BCL2L1,BIRC2,...,Tumor.substage,Sample.ID,Sample.type,Sample.Source,Sample.age.(years),#.of.Replicates,DNA.ID,#.Included.in.CNV.Data.table,RNA.ID,#.Included.in.RNAseq.data.table
0,FD_p01_FH_31_240716,0.013518,0.101543,0.854651,1.838816,-0.448164,0.081020,0.492994,0.230847,-0.540237,...,IIIC,240716,FFPE,FHCRC,15.400000,1,D88,0,R88,0
1,FD_p01_FH_57_372440,0.015467,0.307847,0.767525,2.059126,-0.499973,0.119818,0.535597,0.536939,-0.682869,...,IIIA,372440,FFPE,FHCRC,12.900000,1,D107,0,R107,0
2,FD_p01_MC_13136_42571_C1,0.119737,0.324585,0.880651,1.829320,-0.537364,-0.071418,0.469898,0.446725,-0.656101,...,NaN,42571_C1,FFPE,Mayo,2.900000,1,D193,1,R193,1
3,FD_p01_MC_7053_39955_B1,-0.046495,0.423303,0.789339,2.149668,-0.543133,-0.046186,0.528996,0.353030,-0.660268,...,NaN,39955_B1,FFPE,Mayo,10.030556,1,D177,1,R177,1
4,FD_p01_MC_7301_40353_A3,0.072006,0.487295,0.827629,1.867978,-0.486311,-0.130185,0.652025,0.541812,-0.578166,...,NaN,40353_A3,FFPE,Mayo,8.969444,1,D182,0,R182,1


In [4]:
prots = ['ABCB8', 'ACADSB', 'ACOT7', 'AIFM1', 'ARNT', 'ATM',
       'ATP6V1D', 'BCL2L1', 'BIRC2', 'CA2', 'CAMK2D', 'CARMIL1', 'CASP7',
       'CCDC167', 'CDKN1A', 'CDKN1B', 'CHUK', 'COX6A1', 'COX6C', 'COX7A2',
       'COX7C', 'CPOX', 'EGFR', 'FDX1', 'GLUD1', 'GOLPH3L', 'GPX1', 'GPX4',
       'HACL1', 'HADH', 'IDH1', 'IDI1', 'IL4I1', 'KEAP1', 'LDHA', 'LGALS1',
       'LTA4H', 'MTDH', 'MYD88', 'NBN', 'NCAPH2', 'NDUFB3', 'NFKB1', 'NFKB2',
       'PDK1', 'PECR', 'PLIN2', 'PTGES2', 'RAB25', 'RELA', 'RHOA', 'RIPK1',
       'RPS6KB2', 'S100A10', 'SENP1', 'STAT3', 'TALDO1', 'TFRC', 'TGM2',
       'TOP3B', 'TPMT', 'VCAM1', 'XIAP', 'YWHAG']

In [4]:
# 100 tumors here... 
df['Tumor.response'] = df['Tumor.response'].map({'sensitive': 1, 'refractory': 0})

# Separate features and labels
X = df[prots]  # Replace 'label_column' with your label column name
y = df['Tumor.response']  # Replace 'label_column' with your label column name

# Split the data into an 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

# Initialize and train the XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

# Make predictions on the test set
xgb_pred = xgb_clf.predict(X_test)
xgb_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the XGBoost model
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_roc_auc = roc_auc_score(y_test, xgb_pred_proba)
print(f"XGBoost Accuracy: {xgb_accuracy}")
print(f"XGBoost ROC AUC: {xgb_roc_auc}")

Random Forest Accuracy: 0.8
Random Forest ROC AUC: 0.8511904761904763
XGBoost Accuracy: 0.8
XGBoost ROC AUC: 0.8333333333333334


In [2]:
# try with all markers..?
import os 


df = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_all_prots_oz_merge.csv")
# df['label'] = df['label'].map({'sensitive': 1, 'refractory': 0})
# Separate features and labels


/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (8807,8811) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# re run but with subset and splits.. 

split_folder_path = '/mnt/ncshare/ozkilim/BRCA/splits/HGSOC_multimodal_100'  # Replace with your actual folder path


rf_test_aucs = []
# Loop over split files and create train, val, test subsets
for filename in os.listdir(split_folder_path):
    if filename.endswith('bool.csv'):  # Assuming the files are in CSV format
        print("split")
        split_df = pd.read_csv(os.path.join(split_folder_path, filename))
        # Extract identifiers
        merged_df = pd.merge(df, split_df, left_on='slide_id', right_on='Unnamed: 0')

        # Create train, validation, and test sets
        train_df = merged_df[merged_df['train']]
        val_df = merged_df[merged_df['val']]
        test_df = merged_df[merged_df['test']]

        X_train = train_df[prots]  # Replace 'label_column' with your label column name
        y_train = train_df['label']  # Replace 'label_column' with your label column name

        X_test = test_df[prots]  # Replace 'label_column' with your label column name
        y_test = test_df['label']  # Replace 'label_column' with your label column name


        # Initialize and train the Random Forest classifier
        rf_clf = RandomForestClassifier()
        rf_clf.fit(X_train, y_train)

        # Make predictions on the test set
        rf_pred = rf_clf.predict(X_test)
        rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

        # Evaluate the Random Forest model
        rf_accuracy = accuracy_score(y_test, rf_pred)
        rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
        print(f"Random Forest ROC AUC: {rf_roc_auc}")
        rf_test_aucs.append(rf_roc_auc)
        
    # break
rf_test_aucs = np.array(rf_test_aucs)
print(rf_test_aucs)
print(np.mean(rf_test_aucs))

split
Random Forest ROC AUC: 0.8285714285714286
split
Random Forest ROC AUC: 0.7283950617283951
split
Random Forest ROC AUC: 0.7000000000000001
split
Random Forest ROC AUC: 0.8650793650793651
split
Random Forest ROC AUC: 0.8428571428571427
split
Random Forest ROC AUC: 0.7285714285714285
split
Random Forest ROC AUC: 0.8785714285714286
split
Random Forest ROC AUC: 0.6944444444444444
split
Random Forest ROC AUC: 0.7374999999999999
split
Random Forest ROC AUC: 0.9249999999999999
[0.82857143 0.72839506 0.7        0.86507937 0.84285714 0.72857143
 0.87857143 0.69444444 0.7375     0.925     ]
0.7928990299823633


In [ ]:
# go back to 64.... more corrolated...